Todo:

- udělat to jako skript, co bere vše neoscrapované

Sólo sešit:
- mazat oscrapované soubory

In [2]:
import os
import re
from datetime import datetime
import pandas as pd

In [10]:
with open(os.path.join("downloads/2024-11-01","cd_Brno_Hamburg_D00_2024-11-01_00-32-19.txt"), "r", encoding="utf-8") as file:
    raw_html = file.read()
spojeni = raw_html.split("""<article""")
spojeni = spojeni[1:]
with open(os.path.join("pokusy","pokus_cd.txt"), "w+", encoding="utf-8") as pokus_file:
    pokus_file.write("\n".join(spojeni))

In [12]:
def oscrapuj_cd(slozka, soubor):
    
    with open(os.path.join(slozka, soubor), "r", encoding="utf-8") as file:
        raw_html = file.read()

    oscrapovano = re.search(r'\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2}', soubor).group(0)
    
    spojeni = raw_html.split("""<article""")
    spojeni = spojeni[1:]
    slovniky = []
    for s in spojeni:
        s = s.splitlines()
        slovnik = {}
        slovnik['prodejce'] = "ČD"
        slovnik['vlaky'], slovnik['obsazenost'], slovnik['zpozdeni'] = [], [], []
        slovnik['oscrapovano'] = datetime.strptime(oscrapovano, "%Y-%m-%d_%H-%M-%S")
        den = False
        for radek in s:
            if "text: to(lastTrain($index()))" in radek:
                slovnik['kam'] = radek.split('>')[-2].split('<')[0]
            if "text: from(firstTrain($index()))" in radek:
                slovnik['odkud'] = radek.split('>')[-2].split('<')[0]
            if ("text: journeyDateTextFrom()" in radek) and (den == False):
                den = radek.split('>')[-2].split('<')[0]
            if "text: depTime(firstTrain($index()))" in radek:
                cas_odjezdu = re.search(r"\d{1,2}:\d\d", radek).group(0)
            if "text: buyButtonText()" in radek:
                cena = radek.split('-->')[1].split('<!--')[0].replace(" Kč","")
                try:
                    slovnik['cena'] = int(cena)
                except:
                    pass
            if "Zjistit cenu" in radek:
                slovnik['cena_poznamka'] = "Zjistit cenu"
            if "Cena v dalším kroku" in radek:
                slovnik['cena_poznamka'] = "Cena v dalším kroku"
            if "text: model.trainTypeAndNum" in radek:
                if radek.split('-->')[1].split('<!--')[0] not in slovnik['vlaky']:
                    slovnik['vlaky'].append(radek.split('-->')[1].split('<!--')[0])
            if """<span aria-hidden="true" data-bind="visible: occupancyLevelFull(), text: occupancyLevelText()" style="display: none;">""" in radek:
                slovnik['obsazenost'].append(radek.split('>')[-2].split('<')[0])
            if """<span data-bind="text: timeLength, visible: timeLength != null &amp;&amp; timeLength != ''">""" in radek:
                jizdni_doba = radek.split('>')[-2].split('<')[0].split(":")
                slovnik['jizdni_doba'] = (int(jizdni_doba[0]) * 60) + int(re.search(r'\d\d', jizdni_doba[1]).group(0))
            if """<span data-bind="text: distance, visible: distance != null &amp;&amp; distance != ''" class="mobile-hidden">""" in radek:
                vzdalenost = radek.split('>')[-2].split('<')[0].replace(" km","")
                try:
                    slovnik['vzdalenost'] = int(vzdalenost)
                except:
                    pass
            if """<span class="icon icon-bus" data-bind="ifnot: isLegend, visible: !isLegend &amp;&amp; icoSrc, css: icoSrc, attr: {title: desc, 'aria-label': desc}" title="Náhradní autobusová doprava" aria-label="Náhradní autobusová doprava"></span>""" in radek:
                slovnik['nahradni_bus'] = True
            if """<span aria-hidden="true" data-bind="text: delayText()">""" in radek:
                if "+" in radek:
                    try:
                        slovnik['zpozdeni'].append(int(radek.split('+')[1].strip().split(' ')[0]))
                    except:
                        slovnik['zpozdeni'].append('chyba')
                elif "><" in radek:
                    slovnik['zpozdeni'].append(0)
            if "Místenka zdarma" in radek:
                slovnik['mistenka_zdarma'] = True
        slovnik['den'] = den.split(' ')[0].lower().strip()
        slovnik['odjezd'] = datetime.strptime('-'.join(den.split(' ')[1].split('.')[::-1]) + " " + cas_odjezdu, "%Y-%m-%d %H:%M")
        slovnik['predstih'] = slovnik['odjezd'] - slovnik['oscrapovano']
        if len(slovnik['vlaky']) == 1:
            slovnik['vlaky'] = [slovnik['vlaky'][0]]
            try:
                slovnik['obsazenost'] = [slovnik['obsazenost'][0]] ## todo: obsazenost se přidává n+1×, toto je laciný workaround pro většinu případů 
            except:
                pass
        slovnik['zpozdeni'] = slovnik['zpozdeni'][1::2]
        slovnik['prestupy'] = len(slovnik['vlaky']) - 1
        slovnik['prostredek'] = 'vlak'
        slovniky.append(slovnik)
    return [x for x in slovniky if x != None]

In [18]:
def oscrapuj_cd(slozka, soubor):
    # Pre-compile regular expressions for better performance
    date_pattern = re.compile(r'\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2}')
    time_pattern = re.compile(r'\d{1,2}:\d\d')
    
    with open(os.path.join(slozka, soubor), "r", encoding="utf-8") as file:
        raw_html = file.read()

    # Extract date once instead of per iteration
    oscrapovano = date_pattern.search(soubor).group(0)
    oscrapovano_dt = datetime.strptime(oscrapovano, "%Y-%m-%d_%H-%M-%S")
    
    # Split only once and store results
    spojeni = raw_html.split("<article")[1:]
    
    # Define common strings as constants to avoid string creation in loop
    COMMON_STRINGS = {
        'to_train': 'text: to(lastTrain($index()))',
        'from_train': 'text: from(firstTrain($index()))',
        'journey_date': 'text: journeyDateTextFrom()',
        'dep_time': 'text: depTime(firstTrain($index()))',
        'buy_button': 'text: buyButtonText()',
        'train_type': 'text: model.trainTypeAndNum',
        'occupancy': '<span aria-hidden="true" data-bind="visible: occupancyLevelFull(), text: occupancyLevelText()"',
        'time_length': '<span data-bind="text: timeLength, visible: timeLength != null &amp;&amp; timeLength != \'\'"',
        'distance': '<span data-bind="text: distance, visible: distance != null &amp;&amp; distance != \'"',
        'delay': '<span aria-hidden="true" data-bind="text: delayText()">'
    }
    
    slovniky = []
    
    for s in spojeni:
        # Initialize dictionary with default values
        slovnik = {
            'prodejce': "ČD",
            'vlaky': [],
            'obsazenost': [],
            'zpozdeni': [],
            'oscrapovano': oscrapovano_dt,
            'prostredek': 'vlak'
        }
        
        lines = s.splitlines()
        den = None
        cas_odjezdu = None
        
        for radek in lines:
            # Use dictionary lookup instead of multiple if statements
            if COMMON_STRINGS['to_train'] in radek:
                slovnik['kam'] = radek.split('>')[-2].split('<')[0]
            elif COMMON_STRINGS['from_train'] in radek:
                slovnik['odkud'] = radek.split('>')[-2].split('<')[0]
            elif COMMON_STRINGS['journey_date'] in radek and not den:
                den = radek.split('>')[-2].split('<')[0]
            elif COMMON_STRINGS['dep_time'] in radek:
                cas_odjezdu = time_pattern.search(radek).group(0)
            elif COMMON_STRINGS['buy_button'] in radek:
                cena = radek.split('-->')[1].split('<!--')[0].replace(" Kč","")
                try:
                    slovnik['cena'] = int(cena)
                except ValueError:
                    pass
            elif COMMON_STRINGS['train_type'] in radek:
                vlak = radek.split('-->')[1].split('<!--')[0]
                if vlak not in slovnik['vlaky']:
                    slovnik['vlaky'].append(vlak)
            elif COMMON_STRINGS['occupancy'] in radek:
                slovnik['obsazenost'].append(radek.split('>')[-2].split('<')[0])
            elif COMMON_STRINGS['time_length'] in radek:
                jizdni_doba = radek.split('>')[-2].split('<')[0].split(":")
                slovnik['jizdni_doba'] = (int(jizdni_doba[0]) * 60) + int(jizdni_doba[1][:2])
            elif COMMON_STRINGS['distance'] in radek:
                try:
                    slovnik['vzdalenost'] = int(radek.split('>')[-2].split('<')[0].replace(" km",""))
                except ValueError:
                    pass
            elif "Náhradní autobusová doprava" in radek:
                slovnik['nahradni_bus'] = True
            elif COMMON_STRINGS['delay'] in radek:
                if "+" in radek:
                    try:
                        slovnik['zpozdeni'].append(int(radek.split('+')[1].strip().split(' ')[0]))
                    except ValueError:
                        slovnik['zpozdeni'].append('chyba')
                elif "><" in radek:
                    slovnik['zpozdeni'].append(0)
            elif "Místenka zdarma" in radek:
                slovnik['mistenka_zdarma'] = True
                
        if den and cas_odjezdu:
            slovnik['den'] = den.split(' ')[0].lower().strip()
            date_str = '-'.join(den.split(' ')[1].split('.')[::-1])
            slovnik['odjezd'] = datetime.strptime(f"{date_str} {cas_odjezdu}", "%Y-%m-%d %H:%M")
            slovnik['predstih'] = slovnik['odjezd'] - slovnik['oscrapovano']
        
        # Post-processing
        if len(slovnik['vlaky']) == 1 and slovnik['obsazenost']:
            slovnik['obsazenost'] = [slovnik['obsazenost'][0]]
        
        slovnik['zpozdeni'] = slovnik['zpozdeni'][1::2]
        slovnik['prestupy'] = len(slovnik['vlaky']) - 1
        
        slovniky.append(slovnik)
    
    return [x for x in slovniky if x]

In [20]:
kam = "data" 
os.makedirs(kam, exist_ok=True)
hotove = [y for y in os.listdir(kam) if y[0:3] == "cd_"]
# hotove = []
for x in os.listdir("downloads"):
    nazev_souboru = "cd_" + x + ".parquet"
    if nazev_souboru not in hotove:
        print(nazev_souboru)
        den = []
        cd = [y for y in os.listdir(f"downloads/{x}") if "cd_" in y] 
        print(f"{x}: {len(cd)}")
        for y in cd:
            den = den + oscrapuj_cd(f"downloads/{x}",y)
        df_den = pd.DataFrame(den).sort_values(by="oscrapovano").reset_index(drop=True)
        df_den.to_parquet(os.path.join(kam,nazev_souboru))

cd_2024-11-10.parquet
2024-11-10: 777
